# GCN Code Exploration

## Undestanding the Data

### File Formats

In [188]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys

def parse_index_file(filename):
    """Read index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def sample_mask(idx, l):
    """N-hot encoding. idx conatains multiple indices, 
    so the output contains multiple elements that are set to True"""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def load_data(dataset_str):
    """
    Loads input data from gcn/data directory

    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.

    All objects above must be saved using python pickle module.

    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    
    print("dataset_str: {}".format(dataset_str))
    
    # read all the pickled files
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        data_file = "data/ind.{}.{}".format(dataset_str, names[i])
        print("reading: ", data_file)
        with open(data_file, 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    
    print("\nGraph:")
    for i in range(10):
        print("Node {}: {}".format(i, graph[i]))
        
    # Read index file
    index_file = "data/ind.{}.test.index".format(dataset_str)
    test_idx_reorder = parse_index_file(index_file)
    test_idx_range = np.sort(test_idx_reorder)

    # stack up 2 csr_matrix objects and Convert this matrix to List of Lists format.
    features = sp.vstack((allx, tx)).tolil() 
    print("\nFeature Vectors:")
    print("Number of Features: {}".format(x.shape[1]))
    print("Number of Training Nodes: {}".format(x.shape[0]))
    print("allx.shape: {} - feature vectors of all training nodes, including labeled and unlabeled (superset of x)".format(allx.shape))
    print("tx.shape:   {} - feature vectors of the test nodes".format(tx.shape))
    print("features:   {} = allx{}  +   tx{}".format(features.shape, allx.shape, tx.shape))
    
    # reorder the feature vectors to match the node order used in the graph matrix
    # assign the range of rows from the features matrix that contain all the test 
    # feature vectors to their correct positions in the feature matrix
    features[test_idx_reorder, :] = features[test_idx_range, :]
    
    # create the adjacency matrix from the graph object
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))
    print("\nLabels:")
    print("Number of Classes: {}".format(y.shape[1]))
    print("Number of Training Nodes: {}".format(y.shape[0]))
    print("Number of Training Nodes per Class: {}".format(y.shape[0]//y.shape[1]))
    print("y.shape:       {} - one-hot labels of labeled training nodes".format(y.shape))
    print("ally.shape:   {} - labels of training nodes, including labeled and unlabeled (superset of y)".format(ally.shape))
    print("ty.shape:     {} - labels of the test nodes".format(ty.shape))
    print("labels:       {} = ally{} + ty{}".format(labels.shape, ally.shape, ty.shape))

    # reorder the labels to match the node order used in the graph matrix ??
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    print("\nfile: ", index_file)
    print("Reorder test feature vectors: ")
    print("   test_idx_reorder:  {} ... {}".format(test_idx_reorder[:5], test_idx_reorder[-5:]))
    print("   len(test_idx_reorder): {} - range: [{} .. {}] - indices of test nodes in the graph".format(len(test_idx_reorder), min(test_idx_reorder), max(test_idx_reorder)))
    print(" - The indices map from the sequential order in which vectors were stored in the in tx and ty files")
    print("   to a position according to the ID (index) used to identify them in the graph dictionary.")
    print("   This is a reshuffle of the test vectors within their own row range in the 'features' matrix [{} .. {}]".format(min(test_idx_reorder), max(test_idx_reorder)))
    print("\n   test_idx_range:  {} ... {}".format(test_idx_range[:5], test_idx_range[-5:]))
    print("   len(test_idx_range): {} - range: [{} .. {}] - indices of test nodes in the graph".format(len(test_idx_range), min(test_idx_range), max(test_idx_range)))
    
    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)
    print("\nSplitting the dataset into train, val and test subsets:")
    print("idx_train: range=[{}, {}] len={}".format(min(idx_train), max(idx_train), len(idx_train)))
    print("idx_val:   range=[{}, {}] len={}".format(min(idx_val), max(idx_val), len(idx_val)))
    print("idx_test:  range=[{}, {}] len={}".format(min(idx_test), max(idx_test), len(idx_test)))

    # Create mask arrays (N-hot encoding)
    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])
    print("\nCreate one boolean mask array (N-hot encoding) for each subset:")
    print("train_mask: shape={}   number of True={}   \tnumber of False={}".format(train_mask.shape, train_mask.sum(), sum([1 if not x else 0 for x in train_mask])))
    print("val_mask:   shape={}   number of True={}   \tnumber of False={}".format(val_mask.shape, val_mask.sum(), sum([1 if not x else 0 for x in val_mask])))
    print("test_mask:  shape={}   number of True={}   \tnumber of False={}".format(test_mask.shape, test_mask.sum(), sum([1 if not x else 0 for x in test_mask])))

    # create an zero-value labels matrix for each subset
    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    print("\ncreate an zero-value labels matrix for each subset")
    
    # copy the labels of each subset into each subsets' label matrix
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]
    print("copy the labels of each subset into each subsets' label matrix:")
    print("y_train: shape={}".format(y_train.shape))
    print("y_val:   shape={}".format(y_val.shape))
    print("y_test:  shape={}".format(y_test.shape))

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask

In [189]:
# config
options = {}
options['dataset'] = 'pubmed' # 'cora'] = 'citeseer'] = 'pubmed'
options['model'] = 'gcn' # 'gcn'] = 'gcn_cheby'] = 'dense'
options['learning_rate'] = 0.01
options['epochs'] = 200
options['hidden1'] = 16
options['dropout'] = 0.5
options['weight_decay'] = 5e-4
options['early_stopping'] = 10
options['max_degree'] = 3

In [190]:
# Load data
# from utils import load_data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(options['dataset'])

dataset_str: pubmed
reading:  data/ind.pubmed.x
reading:  data/ind.pubmed.y
reading:  data/ind.pubmed.tx
reading:  data/ind.pubmed.ty
reading:  data/ind.pubmed.allx
reading:  data/ind.pubmed.ally
reading:  data/ind.pubmed.graph

Graph:
Node 0: [14442, 1378, 1544, 6092, 7636]
Node 1: [10199, 8359, 2943]
Node 2: [11485, 15572, 10471]
Node 3: [8249]
Node 4: [14044]
Node 5: [1312, 12968]
Node 6: [17284, 8661, 3150, 18614, 7296, 2216, 8981, 13656, 6572, 3509, 13655, 12098, 7691, 9232, 7335, 4464, 2128, 16720, 767, 6697, 18121, 10265]
Node 7: [15577, 14688, 17955, 18425, 6242, 2343, 17354, 14754, 5564, 2019, 19376, 1568, 14843, 1588, 4058, 10243, 8335]
Node 8: [3157]
Node 9: [11186, 7875, 221, 12664, 1456, 7956, 8915, 19447, 16622]

Feature Vectors:
Number of Features: 500
Number of Training Nodes: 60
allx.shape: (18717, 500) - feature vectors of all training nodes, including labeled and unlabeled (superset of x)
tx.shape:   (1000, 500) - feature vectors of the test nodes
features:   (19717,

In [191]:
print("The graph has {} nodes and {} edges.\n\n{}\n".format(adj.shape[0], adj.todense().sum(), adj.shape))
print(adj.todense()[0:20,0:20])

The graph has 19717 nodes and 88651 edges.

(19717, 19717)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [192]:
print("There are {} nodes and {} classes.".format(y_train.shape[0], y_train.shape[1]))

There are 19717 nodes and 3 classes.


In [193]:
print(len(y_train), len(y_val), len(y_test))
print(y_train[:3])
print(y_val[:3])
print(y_test[:3])

19717 19717 19717
[[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


# Understanding the Math
* * *

In [194]:
import numpy as np
import scipy.sparse as sp

### Adjacency Matrix Normalization

In [184]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    print("adj=\n", adj.todense())
    rowsum = np.array(adj.sum(1))
    print('adj.sum(1)=\n', rowsum)
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    print('d_inv_sqrt=\n', d_inv_sqrt)
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    print('d_mat_inv_sqrt=\n', d_mat_inv_sqrt.todense())
    print('adj.dot(d_mat_inv_sqrt)=\n', adj.dot(d_mat_inv_sqrt).todense())
    print('adj.dot(d_mat_inv_sqrt).transpose()=\n', adj.dot(d_mat_inv_sqrt).transpose().todense())
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

In [185]:
adj = np.array([[0, 1], [1, 3]])
adj.flatten()

array([0, 1, 1, 3])

In [186]:
normalized_adj = normalize_adj(adj)

adj=
 [[0 1]
 [1 3]]
adj.sum(1)=
 [[1]
 [4]]
d_inv_sqrt=
 [1.  0.5]
d_mat_inv_sqrt=
 [[1.  0. ]
 [0.  0.5]]
adj.dot(d_mat_inv_sqrt)=
 [[0.  0.5]
 [1.  1.5]]
adj.dot(d_mat_inv_sqrt).transpose()=
 [[0.  1. ]
 [0.5 1.5]]


In [187]:
print('normalized_adj=\n', normalized_adj.todense())

normalized_adj=
 [[0.   0.5 ]
 [0.5  0.75]]


# Writing data to Normalized Files
* * *

In [196]:
import pickle as pkl
import networkx as nx

def write_data(dataset, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, graph):
    """
    Write input data to gcn/data_out directory

    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instans
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.

    All objects above must be saved using python pickle module.

    :param dataset_str: Dataset name
    :return: None
    """
    
    # write all the pickled files
    objects_dic = {
        'x': x, 
        'y': y, 
        'tx': tx, 
        'ty': ty, 
        'allx': allx, 
        'ally': ally, 
        'graph': graph
    }
    
    objects = []
    for i in range(len(names)):
        data_file = "data_out/ind.{}.{}".format(dataset, names[i])
        print("writing: ", data_file)
        with open(data_file, 'wb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.dump(f, encoding='latin1'))
            else:
                objects.append(pkl.dump(f))

    print("Done.")

In [ ]:
dataset = "pubmed"
write_data(dataset, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, graph)

# Plan
* * *

1. Develop a write_data() function to save the graph data into the file format expected by Kipf source code.
* Convert SROIE labels into a graph, following the method described in Lohani's paper (Lohani et al., 2019, An Invoice Reading System Using a Graph Convolutional Network.)
    - SROIE Features: words inside the bounding box
    - SROIE Categories: 
        - company_value
        * date_title
        * date_value
        * address_title
        * address_value
        * total_title
        * total_value
* Write SROIE graph in normalized format as expected by Kipf's source code. Call the write_data() function that we developed in this notebook.
* Run Kipf's source code on SROIE data. GCN code trains the model, runs inference on the test nodes and measure accuracy
* Write results to google doc, make corrections, feedback, etc.
    - Problems with Kipf method:
        - the reference networks are 1 single big graph, Receipts are one separate graph each.
        * In a reference network we can afford to have 20 labeled examples per class, in a receipt there is only one labeled field of each class
        * linked nodes in a reference network are alike (papers have the same topics). The fields in a recipt that are close by are not necesarily similar.
        * one notable weakness of Spectral Convolution methods is that they are only useful for learning patterns on fixed graphs, and can’t effectively learn on or take into account cases where each observed graph has a different connection structure. This is because each set of calculated eigenvalues is only coherent to apply to the specific graph it was calculated on.
    * Alternative Approach: Message-Passing Neural Networks (MPNN)
        

In [15]:
# SROE dataset
dataset_str = "0325updated.task1train(626p)"
task1train_dir = "/home/adrian/as/blogs/nanonets/datasets/SROIE2019-20191212T043930Z-001/SROIE2019/{}/".format(dataset_str)


# Ignore
* * * 

In [13]:
from scipy.sparse.csr import csr_matrix

def ducuments_to_csr_matrix(docs, debug=False):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in docs:
        for term in d:

            # vocabulary: each word is inserted as key in the dictionary (if it doesnt exist)
            # and the value is the index which is given by the length of the dictionary at the moment of insertion
            index = vocabulary.setdefault(term, len(vocabulary))

            # indices: a flat list of indices to each word of each document, in document input order.
            indices.append(index)

            # data: a list of 1's
            data.append(1)

        # indptr: index to index. It contains one pointer per document, 
        # is a pointer to the pointer to the first word of a document in the indices array.
        indptr.append(len(indices))

    x = csr_matrix((data, indices, indptr), dtype=int).toarray()
    
    if debug:
        print("Details to understand the csr_matrix structure.")
        print("\nvocabulary={}\n".format(vocabulary))
        print("docs={}".format(docs))
        print('\nindices: a flat list of indices to each word of each document, in document input order.')
        print("indices={}".format(indices))
        print("\nindptr: index to index. Contains pointers to the positions in the indices array that point to the first word of each document.")
        print("indptr={}".format(indptr))
        print("csr_matrix=\n{}".format(x))
    
    return x

In [198]:
# Example to understand the csr_matrix structure.
docs = [["hello", "world", "hello"], ["goodbye", "cruel", "world"]]
x = ducuments_to_csr_matrix(docs, True)
x

Details to understand the csr_matrix structure.

vocabulary={'hello': 0, 'world': 1, 'goodbye': 2, 'cruel': 3}

docs=[['hello', 'world', 'hello'], ['goodbye', 'cruel', 'world']]

indices: a flat list of indices to each word of each document, in document input order.
indices=[0, 1, 0, 2, 3, 1]

indptr: index to index. Contains pointers to the positions in the indices array that point to the first word of each document.
indptr=[0, 3, 6]
csr_matrix=
[[2 1 0 0]
 [0 1 1 1]]


array([[2, 1, 0, 0],
       [0, 1, 1, 1]])

## Reformating SROIE labels files as GCN input files
* * *
as scipy.sparse.csr.csr_matrix:
- ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
* ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
* ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances;

as numpy.ndarray:
- ind.dataset_str.y => the one-hot labels of the labeled training instances;
* ind.dataset_str.ty => the one-hot labels of the test instances;
* ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx;


as collections.defaultdict:
- ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]};


as list:
- ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting.


All objects above must be saved using python pickle module.

#### Definitions:
- param dataset_str: Dataset name
* training instance:
* feature vector:
* labels:

In [1]:
# pip install bpemb
from bpemb import BPEmb

# load English BPEmb model with default vocabulary size (10k) and 50-dimensional embeddings
bpemb_en = BPEmb(lang="en", dim=100)

# apply English BPE subword segmentation model
print(bpemb_en.encode("medfordshire bimodal distribution"))

# Decode byte-pair-encoded text: (reverse segmentation)
print(bpemb_en.decode(['▁this', '▁is', '▁an', 'arch', 'ism']))

# The encode-decode roundtrip is lossy:
print("101", bpemb_en.decode(bpemb_en.encode("101")))

# Byte-pair encode text into IDs for performing an embedding lookup:
ids = bpemb_en.encode_ids("anarchism")
print(ids)
print(bpemb_en.decode_ids(ids))
    
# load Chinese BPEmb model with vocabulary size 100k and default (100-dim) embeddings
bpemb_zh = BPEmb(lang="zh", vs=100000)
# apply Chinese BPE subword segmentation model
bpemb_zh.encode("这是一个中文句子")  # "This is a Chinese sentence."


# Byte-pair encode and embed text:
bpemb_en.embed("anarchism").shape

['▁med', 'ford', 'shire', '▁b', 'im', 'od', 'al', '▁distribution']
this is anarchism
101 000
[32, 863, 679]
anarchism


(3, 100)

In [2]:
for idx, subw, emb in zip(bpemb_en.encode_ids("bimodal"), bpemb_en.encode("bimodal"),  bpemb_en.embed("bimodal")):
    print(idx, subw, emb)

20 ▁b [ 3.57920e-02 -1.91620e-01 -3.95140e-02  1.96240e-01  1.98990e-02
 -2.07640e-02 -1.94786e-01  1.29007e-01  1.29045e-01 -8.19000e-04
 -2.93072e-01  1.49086e-01  4.33130e-02  1.55525e-01  2.18574e-01
 -1.72513e-01  3.01457e-01  1.73882e-01 -3.16232e-01  1.63880e-01
 -8.40740e-02  4.31424e-01 -1.23822e-01 -3.04110e-01  3.77727e-01
 -3.65180e-02 -9.24350e-02 -1.11603e-01  7.34410e-02  2.14299e-01
 -2.24924e-01  1.16902e-01 -4.04236e-01  1.18613e-01 -1.08001e-01
  4.63670e-02  1.22694e-01 -1.04957e-01 -1.18560e-01  1.98746e-01
  7.80520e-02 -4.80398e-01  2.87600e-02  2.78143e-01 -1.37563e-01
  1.94820e-02 -7.68560e-02  1.78576e-01 -6.17260e-02 -6.47780e-02
 -6.23970e-02  4.24980e-02  6.49300e-02  4.02810e-02  8.61500e-03
 -2.30266e-01 -1.78978e-01 -1.25859e-01 -8.23780e-02 -1.12904e-01
 -2.26400e-03 -1.50030e-02 -8.56650e-02 -7.23310e-02 -8.12173e-01
  2.97250e-02  8.44700e-03  2.14885e-01  1.92571e-01 -2.50996e-01
  1.68115e-01 -1.80000e-05  5.70470e-02  2.05556e-01  4.13103e-01
  2.